In [ ]:
import pandas as pd
from geopy.geocoders import GoogleV3
import re
import numpy as np

In [ ]:
geolocator = GoogleV3(api_key='api')
cols = ['id', 'school_name', 'school_address', 'suburb', 'postcode', 'sector', 'school_type', 'school_lat', 'school_lon']

In [ ]:
def get_coord_and_postcode(school_name):
    '''
    params:
        
    return:
        
    '''
    location = geolocator.geocode(school_name)
    if location is not None:
        postcode = location[0].split(',')[-2].split(' ')[-1]
        school_lat = location[1][0]
        school_lon = location[1][1]
        # print(postcode, school_lat, school_lon)
        return (postcode, school_lat, school_lon)
    else:
        return None
    

In [ ]:
def get_postcode(coord):
    '''
    params:
        
    return:
        
    '''
    postcode = geolocator.reverse(coord)[0][0].split(',')[-2].split(' ')[-1]
    return postcode

In [ ]:
def get_school_address(school_name):
    '''
    params:
        
    return:
        
    '''
    location = geolocator.geocode(school_name)
    if location is not None:
#         print('1. ', school_name)
        address = location.address.split(',', 1)[0]
        return address
    else:
        print('2. ', school_name)
        return 'N/A'

## ACT_School_locations

In [ ]:
ACT_School = pd.read_csv('datasets/ACT_School_Locations.csv', dtype={'Location 1':str})
ACT_School.head(2)

In [ ]:
ACT_School.rename(columns={'School Name':'school_name', 'Street Address':'school_address', 'Sector':'sector','Suburb':'suburb', 'Type':'school_type'}, inplace=True)

In [ ]:
ACT_School['postcode'] = ''
ACT_School['id'] = range(1, len(ACT_School)+1)
ACT_School['school_lat'] = 0.
ACT_School['school_lon'] = 0.

In [ ]:
ACT_School['Location 1'].fillna('nan', inplace=True)

In [ ]:
for i in range(len(ACT_School)):
    if ACT_School['Location 1'][i] != 'nan':
        ACT_School['school_lat'].values[i] = re.findall(r'[(](.*?)[)]', ACT_School['Location 1'][i])[0].split(',')[0]
        ACT_School['school_lon'].values[i] = re.findall(r'[(](.*?)[)]', ACT_School['Location 1'][i])[0].split(',')[1]
    else:
        continue

In [ ]:
ACT_School.drop(columns=['Location 1'], inplace=True)

In [ ]:
ACT_School = ACT_School.reindex(columns=cols)

In [ ]:
ACT_School.head(2)

In [ ]:
for i in range(len(ACT_School)):
    if ACT_School['school_lat'].values[i] != 0. and ACT_School['school_lon'].values[i] != 0.:
#         print(i, 'a')
        coord = str(ACT_School['school_lat'].values[i]) + ' ' + str(ACT_School['school_lon'].values[i])
        postcode = get_postcode(coord)
        ACT_School['postcode'].values[i] = postcode
    else:
        school_name = ACT_School['school_name'][i] + ' ACT Australia'
#         print(i, school_name)
        res = get_coord_and_postcode(school_name)
        if res != None:
            ACT_School['postcode'].values[i] = res[0]
            ACT_School['school_lat'].values[i] = res[1]
            ACT_School['school_lon'].values[i] = res[2]
        else:
            continue

In [ ]:
ACT_School

In [ ]:
ACT_School.loc[ACT_School['school_name'] == 'Hall Preschool - Gold Creek School', 'school_lat'] = -35.1678009
ACT_School.loc[ACT_School['school_name'] == 'Hall Preschool - Gold Creek School', 'school_lon'] = 149.0695049
ACT_School.loc[ACT_School['school_name'] == 'Hall Preschool - Gold Creek School', 'postcode'] = 2618

In [ ]:
ACT_School.loc[ACT_School['school_name'] == 'Hall Preschool - Gold Creek School']

In [ ]:
ACT_School.to_csv('datasets/finished/ACT_School.csv', index=False)

## NSW_nongov_locations-2017

In [ ]:
NSW_nongov = pd.read_csv('datasets/NSW_nongov_locations-2017.csv')
NSW_nongov.head(2)

In [ ]:
NSW_nongov.info()

In [ ]:
NSW_nongov.rename(columns={'street':'school_address', 
                           'Sector':'sector',
                           'level_of_schooling':'school_type', 
                           'latitude':'school_lat', 
                           'longitude':'school_lon',
                           'town_suburb':'suburb'}, 
                  inplace=True)

In [ ]:
NSW_nongov

In [ ]:
NSW_nongov['id'] = range(1, len(NSW_nongov) + 1)

In [ ]:
NSW_nongov = NSW_nongov.reindex(columns=cols)

In [ ]:
NSW_nongov.info()

In [ ]:
NSW_nongov.to_csv('datasets/finished/NSW_nongov.csv', index=False)

## NSW_Public

In [ ]:
NSW_Public = pd.read_csv('datasets/NSW-Public-Schools-Master-Dataset-07032017.csv', dtype={'postcode':str})
# NSW_Public.info()

In [ ]:
NSW_Public.rename(columns={'school_code':'id', 
                           'street':'school_address', 
                           'town_suburb':'suburb', 
                           'level_of_schooling':'school_type', 
                           'latitude':'school_lat', 
                           'longitude':'school_lon'}, 
                  inplace=True)

In [ ]:
NSW_Public = NSW_Public.reindex(columns=cols)
# NSW_Public.info()

In [ ]:
school_name = NSW_Public.loc[NSW_Public['school_lat'].isna() == True, 'school_name']
location = geolocator.geocode(school_name)

NSW_Public.loc[NSW_Public['school_lat'].isna() == True, 'school_lat'] = location.latitude
NSW_Public.loc[NSW_Public['school_lon'].isna() == True, 'school_lon'] = location.longitude

In [ ]:
NSW_Public['suburb'].fillna('', inplace=True)
NSW_Public['postcode'].fillna('', inplace=True)
NSW_Public['sector'].fillna('Goverment', inplace=True)

In [ ]:
for i in range (len(NSW_Public)):

    if NSW_Public['suburb'].values[i] == '':
        coord = (NSW_Public['school_lat'].values[i], NSW_Public['school_lon'].values[i])
        location = geolocator.reverse(coord, exactly_one=True)
        NSW_Public['suburb'].values[i] = ' '.join(re.findall(r'[A-Z][a-z]+', location.address.split(',')[-2]))
        if NSW_Public['postcode'].values[i] == '':
            NSW_Public['postcode'].values[i] = location.address.split(',')[-2].split(' ')[-1]


In [ ]:
NSW_Public.to_csv('datasets/finished/NSW_Public.csv', index=False)

## Queensland_School

In [ ]:
Queensland = pd.read_csv('datasets/queesland school.csv', dtype={'school_type':str, 'postcode':str})
Queensland.info()

In [ ]:
for i in range(len(Queensland)):
    Queensland['school_type'].values[i] = Queensland['Official Low Year Level'].values[i] + ' to ' + Queensland['Official High Year Level'].values[i]

In [ ]:
Queensland['sector'] = Queensland['sector'].fillna('')

In [ ]:
for i in range(len(Queensland)):
    if Queensland['sector'].values[i] == '':
        Queensland['sector'].values[i] = Queensland['Sector'].values[i]

In [ ]:
Queensland.drop(columns=['Official Low Year Level', 'Official High Year Level', 'Sector'], inplace=True)

In [ ]:
Queensland = Queensland.reindex(columns=cols)

In [ ]:
Queensland.head(2)

In [ ]:
Queensland.to_csv('datasets/finished/Queensland.csv', index=False)

## SA_School

In [ ]:
SA_School = pd.read_excel('datasets/SouthAustralia.xlsx')
SA_School.head(2)

In [ ]:
SA_School.rename(columns={'Number':'id', 
                          'Name':'school_name', 
                          'Address':'school_address', 
                          'Suburb':'suburb', 
                          'Post Code':'postcode', 
                          'Latitude':'school_lat',
                          'Longitude':'school_lon',
                          'Subtype Name':'school_type',
                          'Department':'sector'},
                inplace=True)

In [ ]:
SA_School.info()

In [ ]:
SA_School['school_address'].fillna('', inplace=True)

In [ ]:
for i in range(len(SA_School)):
    if SA_School['school_address'].values[i] == '':
        school_name = SA_School['school_name'].values[i] + ' Australia'
        address = get_school_address(school_name)
        SA_School['school_address'].values[i] = address

In [ ]:
for i in range(len(SA_School)):
    if SA_School['sector'].values[i] == 'DEPT':
        SA_School['sector'].values[i] = 'Government'
    else:
        SA_School['sector'].values[i] = 'Non-Government'

In [ ]:
SA_School = SA_School.reindex(columns=cols)

In [ ]:
SA_School.to_csv('datasets/finished/SA_School.csv', index=False)

## Victoria_School

In [ ]:
Victoria = pd.read_csv('datasets/Victoria.csv')
Victoria.head(2)

In [ ]:
Victoria.info()

In [ ]:
Victoria.rename(columns={'Education_Sector':'sector',
                         'School_No':'id',
                         'School_Name':'school_name',
                         'School_Type':'school_type',
                         'Adress_Line_1':'school_address',
                         'Adress_Town':'suburb',
                         'Address_Postcode':'postcode',
                         'X':'school_lon',
                         'Y':'school_lat'},
               inplace=True)

In [ ]:
Victoria = Victoria.reindex()